*1) Data Reduction:*

In [1]:
#!pip install pyarrow
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
df_sb_f = pd.read_csv("data/df_timeseries_en.tsv.gz", compression="infer", sep="\t")
df_sb_f["datetime"] = pd.to_datetime(df_sb_f["datetime"])
df_sb_f.head(5)


,channel,category,datetime,views,delta_views,subs,delta_subs,videos,delta_videos,activity
0,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-03,2.024946e+05,0.000000,650.222222,0.000000,5,0,3
1,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-10,3.940857e+05,191591.111111,1046.000000,395.777778,6,1,1
2,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-17,8.353938e+05,441308.083333,1501.500000,455.500000,6,0,1
3,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-24,1.104577e+06,269183.250000,1750.000000,248.500000,6,0,0
4,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-31,1.284406e+06,179828.600000,2008.300000,258.300000,6,0,0


In [8]:
print("channel count in TimeSeries table:", df_sb_f['channel'].nunique())
df_sb_f["category"].unique()

chanell count in TimeSeries table: 133516


array(['Film and Animation', 'Music', 'Comedy', 'Gaming',
       'Science & Technology', 'Sports', 'Entertainment', 'Education',
       'Nonprofits & Activism', 'People & Blogs', 'Howto & Style',
       'News & Politics', 'Travel & Events', 'Autos & Vehicles',
       'Pets & Animals', nan], dtype=object)

In [9]:
df_sb_f = df_sb_f.loc[df_sb_f["category"].isin(['Nonprofits & Activism', 'People & Blogs', 'News & Politics'])]
print("channel count in TimeSeries table after filteration:", df_sb_f['channel'].nunique())

chanell count in TimeSeries table after filteration: 21163


Now we save the reduced timeseries table

In [10]:
df_sb_f.to_pickle("data/df_timeseries_reduced.tsv.gz")

In [5]:
#too big.. don't fit in memory
#del df_sb_f
#df_vd_f = pd.read_feather("data/yt_metadata_helper.feather")
#df_vd_f["dummmy"] = 1
#df_vd_f.head(5)


In [11]:

df_ch_f = pd.read_csv("data/df_channels_en.tsv.gz", compression="infer", sep="\t")
df_ch_f["join_date"] = pd.to_datetime(df_ch_f["join_date"])
df_ch_f.head(5)


,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [13]:
print("channel count in TimeSeries table", df_ch_f['channel'].nunique())
df_ch_f['category_cc'].unique()

channel count in TimeSeries table 136470


array(['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports',
       'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism',
       'People & Blogs', 'News & Politics', 'Science & Technology',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan],
      dtype=object)

In [15]:
df_ch_f = df_ch_f.loc[df_ch_f["category_cc"].isin(['Nonprofits & Activism', 'People & Blogs', 'News & Politics'])]
print("channel count in TimeSeries table after filteration:", df_ch_f['channel'].nunique())

channel count in TimeSeries table after filteration: 21645


In [16]:
df_ch_f.to_pickle("data/df_channels_reduced.tsv.gz")

In [13]:

count = 0
count_after = 0
chunksize = 10 ** 5
flag=False
for chunk in tqdm(pd.read_json("data/yt_metadata_en.jsonl.gz",lines=True, chunksize=chunksize)):
    count += chunk['display_id'].nunique() 
    chunk = chunk.loc[chunk["categories"].isin(['Nonprofits & Activism', 'People & Blogs', 'News & Politics'])]
    chunk.drop(['crawl_date', 'categories'], axis=1)
    print(chunk.columns)
    break
    if not flag:
        flag=True
        chunk.to_csv('data/yt_metadata_reduced.tsv.gz', index=False)
    else:
        chunk.to_csv('data/yt_metadata_reduced.tsv.gz', index=False, header=False, mode='a')
    count_after += chunk['display_id'].nunique()

print("videos count: ", count)
print("videos count after reduction: ", count_after)


0it [00:02, ?it/s]

Index(['categories', 'channel_id', 'crawl_date', 'description',
       'dislike_count', 'display_id', 'duration', 'like_count', 'tags',
       'title', 'upload_date', 'view_count'],
      dtype='object')
videos count:  100000
videos count after reduction:  0


*2) Data Exploration & Processing:*

In [18]:
columns = ['categories', 'channel_id', 'crawl_date', 'description',
       'dislike_count', 'display_id', 'duration', 'like_count', 'tags',
       'title', 'upload_date', 'view_count']
for chunk in tqdm(pd.read_csv('data/yt_metadata_reduced.tsv.gz', chunksize=10**5, names=columns)):

    # if columns is None:
    #     columns = chunk.columns
    #     print(columns)
    print(chunk.head(5))


0it [00:00, ?it/s]/home/ewais/anaconda3/envs/deca-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (4,6,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
1it [00:01,  1.06s/it]

       categories                channel_id                  crawl_date  \
0      categories                channel_id                  crawl_date   
1  People & Blogs  UCzWrhkg9eK5I8Bm3HfV-unA  2019-10-31 20:19:35.725333   
2  People & Blogs  UCzWmWTPCafQRNl9q9Y53AfQ  2019-11-17 05:07:30.405938   
3  People & Blogs  UCzWmWTPCafQRNl9q9Y53AfQ  2019-11-17 05:07:30.959894   
4  People & Blogs  UCzWmWTPCafQRNl9q9Y53AfQ  2019-11-17 05:07:31.467288   

                                         description  dislike_count  \
0                                        description  dislike_count   
1  Thomas and friends with pirate Salty with skif...            0.0   
2     the original sucked so here's its reawakening.          613.0   
3                                  It's finally time          588.0   
4  this one's for you NEs, enjoy watching the neu...         1177.0   

    display_id  duration  like_count  \
0   display_id  duration  like_count   
1  LZPh8ky-RP8       175         8.0   
2 

2it [00:02,  1.02s/it]

             categories                channel_id                  crawl_date  \
100000  News & Politics  UCrwE8kVqtIUVUzKui2WVpuQ  2019-11-01 23:45:17.390071   
100001  News & Politics  UCrwE8kVqtIUVUzKui2WVpuQ  2019-11-01 23:45:23.173001   
100002  News & Politics  UCrwE8kVqtIUVUzKui2WVpuQ  2019-11-01 23:45:25.839463   
100003  News & Politics  UCrwE8kVqtIUVUzKui2WVpuQ  2019-11-01 23:45:30.296989   
100004  News & Politics  UCrwE8kVqtIUVUzKui2WVpuQ  2019-11-01 23:45:17.021880   

                                              description  dislike_count  \
100000  Subscribe - http://bit.ly/2ofH4S4\n\n• Faceboo...            1.0   
100001  Subscribe - http://bit.ly/2ofH4S4\n\n• Faceboo...            5.0   
100002  BJP's Southern Push' - Exclusive Interview wit...            4.0   
100003  After formation of BJP govt in Tripura, the de...            1.0   
100004  Subscribe - http://bit.ly/2ofH4S4\n\n• Faceboo...            0.0   

         display_id  duration  like_count  \
100000  Ch4

3it [00:03,  1.14s/it]

...

              categories                channel_id  \
1800000  News & Politics  UClmFQfVyDvdtCralM-Oo_mw   
1800001  News & Politics  UClmFQfVyDvdtCralM-Oo_mw   
1800002  News & Politics  UClmFQfVyDvdtCralM-Oo_mw   
1800003  News & Politics  UClmFQfVyDvdtCralM-Oo_mw   
1800004  News & Politics  UClmFQfVyDvdtCralM-Oo_mw   

                         crawl_date  \
1800000  2019-11-20 22:43:25.550747   
1800001  2019-11-20 22:43:26.140733   
1800002  2019-11-20 22:43:26.846843   
1800003  2019-11-20 22:43:28.225142   
1800004  2019-11-20 22:43:28.901466   

                                               description  dislike_count  \
1800000  Jak Mercer of Logan's Alley in Grand Rapids ag...            0.0   
1800001  Saline 2019 defensive back MJ Griffin on trans...            1.0   
1800002  Local garage bands took the stage on Aug. 2, 2...            0.0   
1800003  Thesea Petto's defense attorney Mitchell Foste...            1.0   
1800004  Michigan communities to see new voting machine

20it [00:23,  1.22s/it]

             categories                channel_id                  crawl_date  \
1900000  People & Blogs  UClDR7ENw4yRUo4mitlBG0Mg  2019-11-04 00:06:46.972390   
1900001  People & Blogs  UClDR7ENw4yRUo4mitlBG0Mg  2019-11-04 00:06:47.621740   
1900002  People & Blogs  UClDR7ENw4yRUo4mitlBG0Mg  2019-11-04 00:06:48.217654   
1900003  People & Blogs  UClDR7ENw4yRUo4mitlBG0Mg  2019-11-04 00:06:48.792504   
1900004  People & Blogs  UClDR7ENw4yRUo4mitlBG0Mg  2019-11-04 00:06:46.960670   

                                               description  dislike_count  \
1900000  No it will not be called Jake Orion lol.\n\nTh...            4.0   
1900001  Exclusively for you guys.\n\nHope you all enjo...           93.0   
1900002  This video includes the altercaton and my reac...            5.0   
1900003  Yes, its true and was embarrassing.\n\nThank y...          107.0   
1900004  This is a honest video of 5 things I hate abou...            5.0   

          display_id  duration  like_count  \
1900

20it [00:24,  1.21s/it]


KeyboardInterrupt: 